### NB: Kjøringen av denen filen vil resulatere i en yaml fil på underkant av 4 millioner linjer, med en størrelse på ca. 103 mb. Maks limit på standard git er 100 mb.  Tar ca. 280 minutter å kjøre koden. 


In [1]:
import json
import requests


In [2]:

def hent_data(url):
    # Sender en GET-forespørsel til URL-en
    respons = requests.get(url)

    # Sjekker om forespørselen var vellykket
    if respons.status_code == 200:
        # Returnerer JSON-innholdet i responsen
        return respons.json()
    else:
        # Returnerer en feilmelding hvis forespørselen mislykkes
        return f"Feil i forespørsel: Statuskode {respons.status_code}"

# URL til API-endepunktet
url = "https://data.qa.ssb.no/pxapi2-beta/api/v2/tables?pageSize=100000"

# Kaller funksjonen og skriver ut resultatet
result = hent_data(url)



In [3]:
# Parse the data
data = result['tables']

In [4]:
data

[{'type': 'Table',
  'id': '13760',
  'label': '13760: Arbeidsstyrken, sysselsatte, arbeidsledige og utførte ukeverk, etter kjønn, alder og type justering. Bruddjusterte tall 2006M01-2023M04',
  'description': '',
  'updated': '2023-04-26T06:00:00Z',
  'firstPeriod': '2006M01',
  'lastPeriod': '2023M04',
  'category': 'public',
  'variableNames': ['kjønn',
   'alder',
   'type justering',
   'statistikkvariabel',
   'måned'],
  'links': [{'rel': 'self',
    'hreflang': 'no',
    'href': 'https://data.qa.ssb.no/pxapi2-beta/api/v2/tables/13760?lang=no'},
   {'rel': 'metadata',
    'hreflang': 'no',
    'href': 'https://data.qa.ssb.no/pxapi2-beta/api/v2/tables/13760/metadata?lang=no'},
   {'rel': 'data',
    'hreflang': 'no',
    'href': 'https://data.qa.ssb.no/pxapi2-beta/api/v2/tables/13760/data?lang=no'}]},
 {'type': 'Table',
  'id': '07458',
  'label': '07458: Sysselsetting, arbeidsledighet og utførte ukeverk, etter alder. Brudd- og sesongjustert, kvartalstall 1997K1-2023K1',
  'descr

In [5]:
# Transform data
tableIds = []
for item in data: 
    new_item = {
        "tableID": item["id"],
    }
    tableIds.append(new_item)
tableIds[0:5]

[{'tableID': '13760'},
 {'tableID': '07458'},
 {'tableID': '13618'},
 {'tableID': '13619'},
 {'tableID': '05110'}]

Hent metadata


In [11]:
import requests


def hent_meta_data(id):
    try:
        url = f"https://data.qa.ssb.no/pxapi2-beta/api/v2/tables/{id}/metadata"
        respons = requests.get(url)

        if respons.status_code == 200:
            data = respons.json()
            data.pop("contacts", None)
            data.pop("licence", None)
            data.pop("links", None)
            return data, None  # Returnerer data og None for feil
        else:
            return None, f"Feil i forespørsel: Statuskode {respons.status_code}"

    except requests.exceptions.RequestException as e:
        return None, str(e)

# ID for spesifikk tabell
tabell_id = "13760"

# Kaller funksjonen med den spesifikke ID-en og skriver ut resultatet
result = hent_meta_data(tabell_id)
print(result)

({'language': 'no', 'id': '13760', 'label': '13760: Arbeidsstyrken, sysselsatte, arbeidsledige  og utførte ukeverk for personer 15-74 år, PxcMetaTitleBy kjønn, alder, type justering, statistikkvariabel PxcMetaTitleAnd måned', 'aggregationAllowed': False, 'officalStatistics': True, 'subjectCode': 'al', 'subjectLabel': 'Arbeid og lønn', 'source': 'Statistisk sentralbyrå', 'updated': '2023-04-26T06:00:00Z', 'notes': [{'mandatory': False, 'text': 'Fra og med 24. november 2022 erstatter tabell 13760 tabell 13332 i månedspubliseringene av AKU-tall. Tabell 13760 er bruddjustert og inneholder også tremåneders glidende gjennomsnitt av sesongjusterte månedstall, altså som tabell 13332. Det som er nytt er for det første at den nye tabellen også inneholder sesongjusterte månedstall, altså tallene som danner grunnlaget for de tremåneders glidende gjennomsnittene. For det andre inneholder den trendtall, som representerer den langsiktige tendensen. Trendtallene er glattere enn den sesongjusterte seri

In [7]:
import yaml

# Writing to a YAML file
with open('metaData.yaml', 'w') as file:
    yaml.dump(result, file, allow_unicode=True)

print("Data saved to data.yaml")

Data saved to data.yaml


In [8]:
# import time
# import yaml

# # Samle all metadata
# all_metadata = []

# for item in tableIds:
#     metadata = hent_meta_data(item["tableID"])
#     if isinstance(metadata, dict):  # Sjekk om responsen er en dictionary
#         all_metadata.append(metadata)
#     time.sleep(3) 

# # Skrive all samlet metadata til en YAML-fil
# with open('all_metadata.yaml', 'w') as file:
#     yaml.dump(all_metadata, file, allow_unicode=True)

# print("All metadata saved to all_metadata.yaml")

In [12]:
import time
import yaml

# Last inn eksisterende data fra YAML-fil
try:
    with open('all_metadata.yaml', 'r') as file:
        all_metadata = yaml.safe_load(file) or []
except FileNotFoundError:
    all_metadata = []

# Samle all metadata og logg suksessfulle IDs
suksessfulle_ids = []
for item in tableIds:
    metadata, error = hent_meta_data(item["tableID"])
    if metadata:
        all_metadata.append(metadata)
        suksessfulle_ids.append(item["tableID"])
    else:
        print(f"Feil ved henting av data for ID {item['tableID']}: {error}")
    time.sleep(2)  # Øk ventetiden for å redusere risikoen for rate limiting


# Skriv oppdatert metadata til YAML-fil
with open('all_metadata.yaml', 'w') as file:
    yaml.dump(all_metadata, file, allow_unicode=True)

# Lagre suksessfulle IDs til en egen fil for fremtidig referanse
with open('suksessfulle_ids.yaml', 'w') as file:
    yaml.dump(suksessfulle_ids, file, allow_unicode=True)


Feil ved henting av data for ID 07118: Feil i forespørsel: Statuskode 404
Feil ved henting av data for ID 05431: Feil i forespørsel: Statuskode 404
Feil ved henting av data for ID 05432: Feil i forespørsel: Statuskode 404
Feil ved henting av data for ID 06047: Feil i forespørsel: Statuskode 404
Feil ved henting av data for ID 05433: Feil i forespørsel: Statuskode 404
Feil ved henting av data for ID 06214: Feil i forespørsel: Statuskode 404
Feil ved henting av data for ID 06213: Feil i forespørsel: Statuskode 404
Feil ved henting av data for ID 04472: Feil i forespørsel: Statuskode 404
Feil ved henting av data for ID 08570: Feil i forespørsel: Statuskode 404
Feil ved henting av data for ID 04187: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Feil ved henting av data for ID 07334: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Feil ved henting av data for ID 06542: ('Connection aborted.', ConnectionResetError(54, 'Connection 